In [1]:
import pandas as pd

raw_data = pd.read_csv("IMDB_dataset_final.csv", low_memory=False)

# printing statements to check the variables
raw_x = raw_data.review; # print(raw_x)
raw_y = raw_data.sentiment; # print(raw_y)

number_of_words_in_dic = 37500; 

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
tf.config.set_visible_devices([], 'GPU')


tokenizer = Tokenizer(num_words=number_of_words_in_dic) # to tokenize the words for learning
tokenizer.fit_on_texts(raw_x)
tokenized_sentiments = tokenizer.texts_to_sequences(raw_x) # converting the words to number arrays

vocab_size = len(tokenizer.word_index) + 1

padded_tokenized_sentiments= tf.keras.utils.pad_sequences(tokenized_sentiments, padding="post", maxlen=1000)

In [2]:
import keras
from keras import Sequential # we will be using this for the CNN
from keras import layers
import tensorflow as tf
import os

The code beneath changes every time you run it and hence shows a bad approach to extracting the embeddings

In [3]:
# importing the trained keras cnn
CNN = tf.keras.models.load_model("CNN_Non_Dense")
CNN.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1000, 50)          1875000   
                                                                 
 conv1d (Conv1D)             (None, 998, 32)           4832      
                                                                 
 global_max_pooling1d (Globa  (None, 32)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 10)                330       
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 1,880,173
Trainable params: 1,880,173
Non-trainable params: 0
______________________________________________

Getting the node embedding outputs from the STABLE CNN model

In [11]:
# getting the outputs from the embedding layer of the CNN
embeddings = keras.Model(inputs=CNN.inputs, outputs=CNN.get_layer(name="embedding").output) 

embedding_raw_outputs = embeddings(padded_tokenized_sentiments[0:50000:1])
flatten_layer = keras.layers.Flatten()
embedding_outputs = flatten_layer(embedding_raw_outputs)
print(embedding_outputs)


tf.Tensor(
[[-0.04275493 -0.08479704  0.0292923  ...  0.08790416  0.07556851
  -0.06879054]
 [ 0.00583083 -0.01126311  0.00234818 ...  0.08790416  0.07556851
  -0.06879054]
 [ 0.01972886  0.07676113 -0.08007895 ...  0.08790416  0.07556851
  -0.06879054]
 ...
 [ 0.01972886  0.07676113 -0.08007895 ...  0.08790416  0.07556851
  -0.06879054]
 [ 0.08189929  0.08689225  0.11288299 ...  0.08790416  0.07556851
  -0.06879054]
 [-0.09420051  0.00115355 -0.06794097 ...  0.08790416  0.07556851
  -0.06879054]], shape=(50000, 50000), dtype=float32)


Splitting this big tensor up into train test and validation segments

In [12]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(embedding_outputs, raw_y, random_state=1000, shuffle=True, test_size=0.3)


TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got array([37314, 20848, 16191, ...,  6215,  4695, 42419])